In [1]:
# finance_analysis

In [2]:
import pandas as pd
from IPython.display import display
import re

In [3]:
# Load files
df1 = pd.read_csv("finances_1.csv")
df2 = pd.read_csv("finances_2.csv")
df3 = pd.read_csv("finances_3.csv")
df4 = pd.read_csv("finances_4.csv")

# Combine
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

df.head()

,Date,Description,Amount
0,12/9/2025,Marriott Fort LauderDania Beach FL,275.72
1,12/9/2025,ONLINE PAYMENT - THANK YOU,-3210.88
2,12/7/2025,Sun Princess Folio CSanta Clarita CA,2815.76
3,12/1/2025,Marriott Fort LauderDania Beach FL,375.16
4,11/28/2025,KINDLE SVCS*BB7H10FM888-802-3080 WA,4.99


In [4]:
# --- Clean & standardize ---

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Ensure Amount is numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Drop any rows with missing Date or Amount
df = df.dropna(subset=['Date','Amount'])

# Sort by date
df = df.sort_values('Date')

df.head(10)


,Date,Description,Amount
317,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-3.07
1516,2024-08-01,NPO* WO SMITH MUSIC SC 08/01 PURCHASE HTTPSBLO...,-25.73
315,2024-08-01,SWEETGREEN BTN 07/30 PURCHASE 202-813-9439 VA,-17.55
316,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-6.15
318,2024-08-01,SQ *BALLSTONIAN 07/31 PURCHASE Arlington VA,-5.50
323,2024-08-02,GUEST SERVICES UNIT 302 08/01 PURCHASE WASHING...,-3.07
322,2024-08-02,GUEST SERVICES UNIT 302 08/01 PURCHASE WASHING...,-7.10
321,2024-08-02,TST* BIG BUNS BALLSTON 07/31 PURCHASE 703-276-...,-19.99
319,2024-08-02,Online Banking transfer from CHK 9131 Confirma...,700.00
320,2024-08-02,CHICK-FIL-A #04085 07/31 PURCHASE ARLINGTON VA,-10.88


In [5]:
print("Number of rows:", len(df))
print("\nData types:")
print(df.dtypes)

print("\nDate range:", df['Date'].min(), "→", df['Date'].max())

print("\nSample of 5 random rows:")
display(df.sample(5))


Number of rows: 1739

Data types:
Date           datetime64[ns]
Description            object
Amount                float64
dtype: object

Date range: 2024-08-01 00:00:00 → 2025-12-12 00:00:00

Sample of 5 random rows:


,Date,Description,Amount
1080,2025-03-21,SUPER POLLO 03/19 PURCHASE ARLINGTON VA,-13.74
335,2024-08-05,SQ *BALLSTONIAN 08/03 PURCHASE Arlington VA,-5.50
1389,2025-08-18,TST*ARLINGTON - ANITAS 08/17 PURCHASE 571-290-...,-17.54
1777,2025-11-10,NETFLIX.COM 11/10 PURCHASE LOS GATOS CA,-24.99
797,2024-12-23,FTL TRIP ADVISOR SHOP 12/20 PURCHASE FT. LAUDE...,-8.54


In [6]:
# Filter to analysis window
start_date = pd.Timestamp("2024-08-01")
end_date   = pd.Timestamp("2025-12-12")

df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

print("Rows in filtered dataset:", len(df))
df.head()


Rows in filtered dataset: 1739


,Date,Description,Amount
317,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-3.07
1516,2024-08-01,NPO* WO SMITH MUSIC SC 08/01 PURCHASE HTTPSBLO...,-25.73
315,2024-08-01,SWEETGREEN BTN 07/30 PURCHASE 202-813-9439 VA,-17.55
316,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-6.15
318,2024-08-01,SQ *BALLSTONIAN 07/31 PURCHASE Arlington VA,-5.50


In [7]:
merchant_category = {


    # === Barber ===
    "ZELLE TO CP": "Barber",  
    "ZELLE": "Barber",
    

    # ===== Books =====
    "KINDLE": "Books",

    # ===== Charity/Gift/Donation =====
    "NPO WO SMITH MUSIC SC HTTPSBLOOMERA IN": "Charity/Gift/Donation",
    "NPO WO SMITH MUSIC SC WWW.WOSMITH.O TN": "Charity/Gift/Donation",
    "NPO WO SMITH MUSIC SC": "Charity/Gift/Donation",
    "NPO* WO SMITH MUSIC SC 08/01 PURCHASE HTTPSBLOOMERA IN": "Charity/Gift/Donation",
    
    "CONDOLENCES.COM TUKIOS.COM/WE UT": "Charity/Gift/Donation",
    "CONDOLENCES.COM": "Charity/Gift/Donation",
    
    "ZELLE TO SG": "Charity/Gift/Donation",
    "ZELLE TO FD PK6PAPFPD": "Charity/Gift/Donation",

    # ===== Donations/Colleges/Universities =====
    "COLUMBIA GIFT -- NY": "Donations/Colleges/Universities ",
    "COLUMBIA GIFT HTTPS://WWW.C NY": "Donations/Colleges/Universities ",
    "COLUMBIA ONLINE GIFT": "Donations/Colleges/Universities ",
    "MTSU ADNCEMENT SERVIC -- TN": "Charity / Gift / Donations/Colleges/Universities ",

    "LEWIS UNIVERSITY": "Donations/Colleges/Universities ",
    "LEWIS UNIVERSITY WWW.LEWISU.ED IL": "Donations/Colleges/Universities ",   
    

    # ===== Dining =====
    "TST BIG BUNS BALLSTON": "Dining",
    "- ANITAS": "Dining",
    "A CHICK-FIL-A": "Dining",
    "BALLSTON - TACO BA --": "Dining",
    "BIG BUNS BALLSTON": "Dining",
    "BIGBUNSDAMNGOODBURGERS ORDER.THOMPSO": "Dining",
    "BONGO JA - TN": "Dining",
    "CHEESEKE OLO.COM": "Dining",
    "CORNER BAKERY BETHESDA": "Dining",
    "CRACKER BARREL - TN": "Dining",
    "CREMA DANIA POINTE MOBILE DANIA BEACH FL": "Dining",
    "FIRST WATCH PAT -- FL": "Dining",
    "GRAZIE GRAZIE - BA": "Dining",
    "GUEST SERVICES": "Dining",
    "IHOP --": "Dining",
    "IHOP OLO": "Dining",
    "KABOB GARDEN": "Dining",
    "POPEYES": "Dining",
    "SPICE KRAFT ( ARLIN": "Dining",
    "SUPER POLLO": "Dining",
    "SUPERETTE GOSQ.COM": "Dining",
    "TACO BAMBA TAQUERIA": "Dining",
    "TACO BAMBA TAQUERIA OLO.COM": "Dining",
    "TFK- - IN -": "Dining",
    "TFK- HTTPS://WWW.T": "Dining",
    "TROPIL SMOOTHIE FE": "Dining",
    "TROPIL SMOOTHIE FE -": "Dining",
    "TSTBALLSTON - TACO BAM --": "Dining",
    "TSTGRAZIE GRAZIE - BAL --": "Dining",
    "TSTROCKLANDS BBQ ARLIN": "Dining",
    "TSTROCKLANDS BBQ ARLIN --": "Dining",
    "TSTTEDS BULLETIN BALLS": "Dining",
    "TSTTEDS BULLETIN BALLS --": "Dining",
    "TSTTHE SALT LINE - BAL": "Dining",
    "TSTTORTAS Y TACOS LA C --": "Dining",
    "TSTTURUS BY TIMBER PIZ --": "Dining",
    "TUPELO HONEY": "Dining",
    "UNION KITCHEN BALLS": "Dining",
    "URBAN TANDOOR HTTPSUTANDOOR": "Dining",
    "URBAN TANDOOR UTANDOOR.CO": "Dining",
    "TST BIG BUNS BALLSTON": "Dining",
    "BIGBUNSDAMNGOODBURGERS": "Dining",
    "TST* BIG BUNS BALLSTON": "Dining",
    "KABOB GARDEN": "Dining",
    "SUPER POLLO": "Dining",
    "TST* POPPYSEED RYE FLOW": "Dining",
    "SQ *UNION KITCHEN BALLS": "Dining",
    "TACO BAMBA TAQUERIA": "Dining",
    "GRAZIE GRAZIE": "Dining",
    "TUPELO HONEY": "Dining",
    "ANITAS": "Dining",
    "FIRST WATCH": "Dining",
    "THE SALT LINE": "Dining",
    "SPICE KRAFT": "Dining",
    "POPEYES": "Dining",
    "TST* SUPERETTE": "Dining",
    "TST*GREENHEART JUICE SH": "Dining",
    "TST* BALLSTON - TACO BAM": "Dining",
    "TST*ARLINGTON - ANITAS": "Dining",
    "TST*GRAZIE GRAZIE - BAL": "Dining",
    "TST*THE SALT LINE - BAL": "Dining",
    "BALLSTON - TACO BA": "Dining",
    "CORNER BAKERY": "Dining",
    "BONGO JA": "Dining",
    "CRACKER BARREL": "Dining",
    "CHEESEKE OLO.COM": "Dining",
    "CREMA DANIA POINTE": "Dining",
    "IHOP": "Dining",
    "GUEST SERVICES": "Dining",  
    "OUTBACK HERMITAGE TN": "Dining",
    "POPPYSEED RYE FLOW": "Dining",
    "POPPYSEED RYE FLOW --": "Dining",
    "CHICK-FIL-A": "Dining",
    "SWEETGREEN": "Dining",
    "BALLSTONIAN": "Dining",
    "DUNKIN": "Dining",
    "TST*TEDS BULLETIN BALLS": "Dining",
    "TST*ROCKLANDS BBQ ARLIN": "Dining",
    "TST*TURUS BY TIMBER PIZ": "Dining",
    "THAI TREASURE": "Dining",
    "SUPERETTE": "Dining",
    "CAVA BALLSTON": "Dining",
    "URBAN TANDOOR": "Dining",
    "TROPICAL SMOOTHIE CAFE": "Dining",
    "STARBUCKS": "Dining",
   

    # ===== Electronics =====
    "DELL INC ROUND ROCK TX": "Electronics",
    "DELL - CONSUMER OROUND ROCK": "Electronics",


    # ===== Entertainment/Gambling =====
    "FANDUEL.COM -- NY": "Entertainment/Gambling",
    "DKDRAFTKINGSYUHT MA": "Entertainment/Gambling",
    "LOTTERY --": "Entertainment/Gambling",

    # ===== Financial Services =====
    "WIRE FEE": "Financial Services",
    "WIRE TRANSFER": "Financial Services",
    "UNCHAINED PITAL WWW.UNCHAINED TX": "Financial Services",


    # ===== Groceries =====
    "HARRIS TE -": "Groceries",
    "SP ANCIENT CRUNCH MASACHIPS.COM NJ": "Groceries",
    "KROGER NASHVILLE TN": "Groceries",
    "SPROUTS FARMER MOBILE DANIA BEACH FL": "Groceries",
    "GIANT": "Groceries",
    "WHOLEFDS": "Groceries",
    "TOTAL WINE": "Groceries",

    
    # ===== Healthcare =====
    "CVS/PHARM": "Healthcare",
    "CVS/PHARMACY": "Healthcare",
    "LABORATORY CORPORATION -- NC": "Healthcare",
    "PENTAGON DENTAL OFFICE PENTAGON": "Healthcare",
    "VIRGINIA VISION ASSOCIA": "Healthcare",
    "GASTROENTEROLOGY CENTER": "Healthcare",
    "CVS/PHARMACY": "Healthcare",
    "VHC ARLINGTON HEALTH SY": "Healthcare",
    "PENTAGON DENTAL OFFICE": "Healthcare",
    "GASTROENTEROLOGY CENTER": "Healthcare",
    "LABORATORY CORPORATION": "Healthcare",
    "PHR*FAMILYMEDICINEINFAL": "Healthcare",
    "LABCORP": "Healthcare",
    "VHC ARLINGTON": "Healthcare",

    
    # ===== Insurance =====
    "AAA MEMBERSHIP GAR HTTPS://WWW.A DE": "Insurance",
    "NORTHWESTERN MU DES:ISA PYMENT": "Insurance",
    "ALLIANZ TRAVEL INS": "Insurance",
    "ALLIANZTRAVELINSURAN": "Insurance",
    "ALLIANZ TRAVEL INS RICHMOND": "Insurance",
    "ALLIANZTRAVELINSURANRICHMOND": "Insurance",
    "PRUDENTIAL": "Insurance",
    "STATE FARM": "Insurance",
    "AAA MEMBERSHIP": "Insurance",
    "ALLIANZ": "Insurance",


    # ===== Laundry =====
    "CUPID CLEANERS": "Laundry",
    "CUPID CLEANERS": "Laundry",


    # ===== Shopping =====
    "AMAZON": "Shopping",
    "TARGET STORE T": "Shopping",
    "TARGET ST": "Shopping",
    "TARGET T- FALLS CHURCH": "Shopping",
    "TARGET T- W NASHVILLE TN": "Shopping",
    "WALGREENS STOR HERMITAGE TN": "Shopping",
    "WALGREENS": "Shopping",
    

    # ===== Subscription/Patreon =====
    "PATREON MEMBERSHIP": "Subscription/Patreon",
    "PATREON MEMBERSHIP -": "Subscription/Patreon",
    "PATREON PATREO SAN FRANCISCO": "Subscription/Patreon",
    "PATREON": "Subscription/Patreon",
    

    # ===== Subscription/AI/Software =====
    "MICROSOFTMICR REDMOND WA": "Subscription/AI/Software ",
    "MICROSOFT": "Subscription/AI/Software",
    "N PLUS MOZILLA.ORG": "Subscription/AI/Software ",  
    "MDN PLUS": "Subscription/AI/Software",
    "SL.NORD VPNCO NEW YORK NY": "Subscription/AI/Software ",
    "OPENAI": "Subscription/AI/Software",
    

    # ===== Subscription/Cloud Storage =====
    "APPLE.COM/BILL": "Subscription/Cloud Storage",
    "APPLE COM BILL": "Subscription/Cloud Storage",
    "APPLE.COM/US": "Subscription/Cloud Storage",   
    "GOOGLE *Google One": "Subscription/Cloud Storage",
    "GOOGLE": "Subscription/Cloud Storage ",
    "ADOBE ADOBE --": "Subscription/Cloud Storage",
    "ADOBE INC -": "Subscription/Cloud Storage",
    "ADOBE INC SAN JOSE": "Subscription/Cloud Storage",
    "ADOBE SAN JOSE": "Subscription/Cloud Storage",
    "ADOBE": "Subscription/Cloud Storage",
    "APPLE": "Subscription/Cloud Storage",
    "REMARKABLE": "Subscription/Cloud Storage",
    
   
    # ===== Subscription/Video Streaming =====
    "NETFLIX": "Subscription/Video Streaming ",
    "HULU": "Subscription/Video Streaming ",
    "GOOGLE*YOUTUBEPREMIUM": "Subscription/Video Streaming",
    "GOOGLEYOUTUBEPREMIUM G.CO/HELPPAY": "Subscription/Video Streaming",
    "Google YouTubePremium": "Subscription/Video Streaming",
    "Google YouTube TV": "Subscription/Video Streaming",
    "GOOGLE *YouTube": "Subscription/Video Streaming",
    "FP RUMBLE USA INC -- FL": "Subscription/Video Streaming",
    "FP *RUMBLE USA": "Subscription/Video Streaming",
    "OTT CRITERIONCHANNEL CRITERION.COM NY": "Subscription/Video Streaming",  
    "CRITERIONCHANNEL": "Subscription/Video Streaming",
    "DISNEYPLUS": "Subscription/Video Streaming",
    "DISNEY PLUS": "Subscription/Video Streaming",
       

    # ===== Subscription/Social Media/News =====
    "MEDIUM ANNUAL": "Subscription/Social Media/News",
    "MEDIUM ANNUAL MEDIUM.COM": "Subscription/Social Media/News",
    "GOOGLE *Ground News": "Subscription/Social Media/News",
    "AUDIBLE": "Subscription/Social Media/News",
    

    # ===== Subscription/Apps =====
    "GOOGLE *QFIT": "Subscription/Apps",
    "GOOGLE *MyFitnessPal": "Subscription/Apps",
    "WWW.WOLFRAMALPHA.COM": "Subscription/Apps",
    "WWW.WOLFRAMALP": "Subscription/Apps",
    "WWW.WOLFRAMALP CHAMPAIGN IL": "Subscription/Apps",
    "WWW.WOLFRAMALPHA.COM HTTPSWWW.WOLF IL": "Subscription/Apps",
    "PADDLE.NET* BGDATABALL": "Subscription/Apps",
    

    # ===== Tax Services/Payments =====
    "CHURN'S TAX SERVICE SANTA ANA": "Tax Services/Payments",
    "DEPT TAXATION DES:WITHDRAWAL ID: INDN:ARTEAMUS CRAYTO CO ID: PPD": "Tax Services/Payments",


    # ===== Travel/Transportation =====
    "UBER": "Travel/Transportation",
    "ALAMO RENT-A-CAR": "Travel/Transportation",
    "ALAMO RENT-A-C": "Travel/Transportation",
    
    
    # ===== Travel/Customs/Fees =====
    "USCUSTOMS TRUSTEDTRAVEL -- IN": "Travel/Customs/Fees",

    
    # ===== Travel/Flights =====
    "AMERIN AIRLINES -- TX": "Travel/Flights",
    "AMERIN AIRLINES FORT WORTH TX": "Travel/Flights",
   
    # ===== Travel/Hotel =====
    "AC Hotel By Marriott": "Travel/Hotel",
    "Marriott Fort Lauder": "Travel/Hotel",
    "MARRIOTT": "Travel/Hotel",
    "AC HOTEL BY MARRIOTTSALT LAKE CITY UT": "Travel/Hotel",
    "SUN PRINCESS FOLIO": "Travel/Hotel",
    "AC HOTEL": "Travel/Hotel",
    "MARRIOTT": "Travel/Hotel",
    

    # ===== Travel/Shopping =====
    "FORT LAUDERDALE AIRPOR FORT LAUDERDA FL": "Travel/Shopping",
    "FTL TRIP ADVISOR SHOP FT. LAUDERDAL FL": "Travel/Shopping",
    
    
    # ===== Utility/Internet  =====
    "COMST -COMST": "Utility/Internet",
    "COMST / XFINITY -COMST": "Utility/Internet",
    "COMST / XFINITY --": "Utility/Internet",
    "COMCAST": "Utility/Internet",
    

    # ===== Utility/Phone =====
    "TMOBILE AUTO P BELLEVUE WA": "Utility/Phone",
    "TMOBILE": "Utility/Phone",
    
    
    # ===== Vehicle/Maintenance =====
    "PHAMS R RE": "Vehicle/Maintenance",
    "PHAMS CAR CARE": "Vehicle/Maintenance",
    "SMART KEY EXPS L ANNANDALE": "Vehicle/Maintenance",
    "SQ *SMART KEY EXPRESS L": "Vehicle/Maintenance",
    "PHAMS": "Vehicle/Maintenance",
    

    # ===== Vehicle/Gas =====
    "MAHESH M TANNA": "Vehicle/Gas",
    "MAHESH M TANNA": "Vehicle/Gas",
    "SHELL OIL": "Vehicle/Gas",
    "SHELL": "Vehicle/Gas",
    "JOE'S KWIK MAR": "Vehicle/Gas",
    "-ELEVEN NASHVILLE TN": "Vehicle/Gas",
    
}


In [15]:

# 1. Define your regex pattern mapping (pattern: category)
regex_category_patterns = [
    (r'CHICK[- ]?FIL[- ]?A', "Dining"),
    (r'Patreon', "Subscriptions/Patreon"),
    (r'OPENAI', "Subscriptions/AI/Software"),
    (r'DUNKIN', "Dining"),
    (r'TST\*?TEDS BULLETIN BALLS', "Dining"),
    (r'SQ \*?BALLSTONIAN', "Dining"),
    (r'GIANT', "Groceries"),
    (r'SWEETGREEN', "Dining"),
    (r'WHOLEFDS', "Groceries"),
    (r'NETFLIX', "Subscription/Video Streaming"),
    (r'KINDLE', "Subscriptions/Apps"),
    (r'TROPICAL SMOOTHIE', "Dining"),
    (r'TST\*?ROCKLANDS BBQ ARLIN', "Dining"),
    (r'AMAZON', "Shopping"),
    (r'JOE\'S KWIK MAR', "Gas / Fuel"),
    (r'UBER', "Travel/Transport"),
    (r'ALAMO RENT-A-CAR', "Travel/Car Rental"),
    (r'COMCAST', "Utility/Internet"),
    (r'COMCAST / XFINITY', "Utility/Internet"),
    (r'FASHION CTR PENTAGON', "Shopping"),
    (r'STARBUCKS', "Dining"),
    (r'APPLE', "Subscription/Cloud Storage"),
    (r'Adobe', "Subscriptions/AI/Software"),
    (r'FP \*RUMBLE USA INC', "Subscription/Video Streaming"),
    (r'Audible', "Subscriptions/Apps"),
    (r'Linux New Media', "Subscriptions/AI/Software"),
    (r'PRINCESS CRUISE RES', 'Travel/Cruises'),
    (r'ENCHANTED PRINCESS', 'Travel/Cruises'),
    (r'CHEESECAKE 072 ONLINE.*OLO\.COM', 'Dining'),
    (r'JBAB BOWL SNACK ALOHA', 'Dining'),
    (r'TFK-ARLINGTON VA.*HTTPS://WWW\.T', 'Dining'),
    (r'FASHION CTR PENTAGON CI.*ARLINGTON VA', 'Vehicle/Parking'),
    (r'LINUX NEW MEDIA USA', 'Subscriptions/Social Media/News'),
    (r'NPO\*? WO SMITH MUSIC SC.*BLOOMERA', 'Charity/Gift/Donation'),
    (r'MTSU ADVANCEMENT SERVIC', 'Donations/Colleges/Universities'),
    (r'UNCHAINED CAPITAL', 'Financial Services'),
    (r'FANDUEL\.COM', 'Entertainment/Gambling'),
    (r'VALOTTERY', 'Entertainment/Gambling'),
    (r'AMERICAN AIRLINES', 'Travel/Airlines'),
    (r'TFK-ARLINGTON VA.*HTTPS://WWW\.T', 'Dining'),
    (r'CHEESECAKE.*OLO\.COM', "Dining"),
    (r'FASHION CTR PENTAGON', "Vehicle/Parking"),
    (r'JBAB BOWL SNACK ALOHA', "Dining"),
    (r'LINUX NEW MEDIA', "Subscription/Social Media/News"),
    (r'NPO.*WO SMITH MUSIC', "Charity/Gift/Donation"),
    (r'PRINCESS CRUISE RES', "Travel/Cruises"),
    (r'TFK-ARLINGTON VA', "Dining"),
    (r'MTSU ADVANCEMENT SERVIC', "Donations/Colleges/Universities"),
    (r'UNCHAINED CAPITAL', "Financial Services"),
    (r'FANDUEL\.COM', "Entertainment/Gambling"),
    (r'ARLINGTON METER PARKING', "Vehicle/Parking"),
    (r'FORT LAUDERDALE AIRPOR', "Travel/Shopping"),
    (r'FTL TRIP ADVISOR SHOP', "Travel/Shopping"),
    (r'TARGET T-', "Shopping"),
    (r'7-ELEVEN', "Groceries"), 
    (r'HARRIS TE \d+', "Groceries"),
    (r'TST\*TORTAS Y TACOS LA C', "Dining"),
    (r'KROGER #\d+', "Groceries"),
    (r'OUTBACK', "Dining"),
    (r'FEDE?X', "Shipping/Logistics"),
    (r'DK\*DRAFTKINGS', "Entertainment/Gambling"),
    (r'VA LOTTERY', "Entertainment/Gambling"),
    (r'DELL', "Electronics"),
    (r'SPROUTS FARMER', "Groceries"),
    (r'SP ANCIENT CRUNCH', "Groceries"),
    (r'FIREHOUSE SUBS', "Dining"),
    (r'NATURAL CAFE', "Dining"),
    (r'HELP\.MAX\.COM', "Subscription/Video Streaming"),
    (r'HELP\.HBOMAX\.COM', "Subscription/Video Streaming"),
    (r'SL\.NORD\* VPNCO', "Subscription/AI/Software"),
    (r'USCUSTOMS TRUSTEDTRAVEL', "Travel/Customs/Fees"),
    (r'SPO\*SIMPLYBANHMIQUINCY', "Dining"),
    (r"VA DEPT TAXATION DES:WITHDRAWAL", "Tax Services/Payments"),
    (r"WASHINGTON & L", "Dining"),



]

In [20]:

# List all key phrases/patterns you want to filter out
exclude_patterns = [
    r'Online Banking transfer',
    r'ONLINE PAYMENT - THANK YOU',
    r'Bank of America Credit Card Bill Payment',
    r'BANK OF AMERICA CREDIT CARD Bill Payment',
    r'AMEX Airline Fee Reimbursement',
    r'RENEWAL MEMBERSHIP FEE',
    r'Platinum Digital Entertainment Credit',
    r'Bruin v BANA Class Sttlmt',
    r'BOFA FIN CTR',
    r'BKOFAMERICA ATM',
    r'Online Scheduled Payment',
    r'Online Banking payment to CRD',
    r'FSAREIMBURSEMENT DES:EDI PYMNTS',
    r'ASSOCIATION OF NATIONAL',
    r"AMERICAN EXPRESS DES:ACH PMT"
    r"Payment"
    r"Transfer"
    r"AMERICAN\s+EXPRESS\s+DES:ACH\s+PMT"

]

# Join patterns into a single regex string
exclude_regex = '|'.join(exclude_patterns)

# Remove unwanted rows (case-insensitive match)
df = df[~df['Description'].str.contains(exclude_regex, case=False, regex=True)].copy()


In [21]:
def map_category(description):
    desc = str(description).upper()
    # Direct Lookup
    if desc in merchant_category:
        return merchant_category[desc]
    # Substring Matching
    for key in merchant_category:
        if key in desc:
            return merchant_category[key]
    # Regex Matching
    for pattern, category in regex_category_patterns:
        if re.search(pattern, desc):
            return category
    return "Uncategorized"


In [22]:
df['Category'] = df['Description'].apply(map_category)


In [23]:
# Review uncategorized items (for manual mapping/fixing)
uncategorized = df[df['Category'] == "Uncategorized"]['Description'].unique()
print(f"Number of uncategorized descriptions: {len(uncategorized)}")
print("Uncategorized descriptions:")
for desc in uncategorized:
    print(desc)


Number of uncategorized descriptions: 22
Uncategorized descriptions:
AMERICAN EXPRESS DES:ACH PMT ID:W9250 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W3058 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W9530 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W1204 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W2310 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W7136 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W6426 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W5346 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W4932 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W0726 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W3356 INDN:ARTEAMUS CRAYTON CO ID:XXXXX33497 WEB
AMERICAN EXPRESS DES:ACH PMT ID:W5936 IN

In [24]:
### drop any remainin unwanted rows ###
# Drop all rows where 'Description' contains the AMEX payment string (case-insensitive)
df = df[~df['Description'].str.contains("AMERICAN EXPRESS DES:ACH PMT", case=False, regex=False)].copy()


In [25]:
# Display the first 10 rows
display(df.head(10))

# Display the last 10 rows
display(df.tail(10))

# Optional: show value counts for each category
display(df['Category'].value_counts())


,Date,Description,Amount,Category
317,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-3.07,Dining
1516,2024-08-01,NPO* WO SMITH MUSIC SC 08/01 PURCHASE HTTPSBLO...,-25.73,Charity/Gift/Donation
315,2024-08-01,SWEETGREEN BTN 07/30 PURCHASE 202-813-9439 VA,-17.55,Dining
316,2024-08-01,GUEST SERVICES UNIT 302 07/31 PURCHASE WASHING...,-6.15,Dining
318,2024-08-01,SQ *BALLSTONIAN 07/31 PURCHASE Arlington VA,-5.50,Dining
323,2024-08-02,GUEST SERVICES UNIT 302 08/01 PURCHASE WASHING...,-3.07,Dining
322,2024-08-02,GUEST SERVICES UNIT 302 08/01 PURCHASE WASHING...,-7.10,Dining
321,2024-08-02,TST* BIG BUNS BALLSTON 07/31 PURCHASE 703-276-...,-19.99,Dining
320,2024-08-02,CHICK-FIL-A #04085 07/31 PURCHASE ARLINGTON VA,-10.88,Dining
1517,2024-08-02,Patreon* Membership 08/01 PURCHASE Internet CA,-26.14,Subscription/Patreon


,Date,Description,Amount,Category
1508,2025-12-09,UBER *TRIP 12/08 PURCHASE HELP.UBER.COM CA,-7.00,Travel/Transportation
1507,2025-12-09,UBER *TRIP 12/08 PURCHASE HELP.UBER.COM CA,-21.98,Travel/Transportation
1505,2025-12-09,FIRST WATCH 0280 PAT 12/07 PURCHASE 941-907-98...,-30.04,Dining
1513,2025-12-10,TFK-ARLINGTON VA 12/09 PURCHASE HTTPS://WWW.T VA,-15.72,Dining
1512,2025-12-10,TST* BALLSTON - TACO BA 12/08 PURCHASE 571-777...,-27.21,Dining
1511,2025-12-10,FIRST WATCH 0280 PAT 12/08 PURCHASE 941-907-98...,-23.58,Dining
1801,2025-12-10,NETFLIX.COM 12/10 PURCHASE LOS GATOS CA,-24.99,Subscription/Video Streaming
1514,2025-12-11,TST*SUPERETTE 12/09 PURCHASE 571-312-0204 VA,-13.26,Dining
1802,2025-12-11,THAI TREASURE 12/10 PURCHASE XXX-XX44102 VA,-35.52,Dining
1515,2025-12-12,STARBUCKS XXXXX27282 12/10 PURCHASE 800-782-72...,-25.00,Dining


Category
Dining                              916
Groceries                            80
Subscription/Patreon                 57
Shopping                             51
Insurance                            51
Travel/Transportation                49
Subscription/Cloud Storage           41
Barber                               37
Subscription/Cloud Storage           36
Subscription/AI/Software             30
Books                                30
Laundry                              29
Subscription/Video Streaming         23
Subscription/Video Streaming         19
Vehicle/Gas                          18
Charity/Gift/Donation                18
Utility/Phone                        16
Utility/Internet                     16
Healthcare                           14
Subscription/Social Media/News       12
Travel/Cruises                        7
Subscription/Apps                     6
Donations/Colleges/Universities       6
Travel/Airlines                       6
Financial Services             

In [26]:
# Export the cleaned and categorized DataFrame to CSV
df.to_csv("financial_transactions_Aug2024-Dec2025.csv", index=False)


In [27]:
import pandas as pd

# Ensure 'Amount' is numeric, take absolute value for spending analysis
df['Spend'] = df['Amount'].abs()

# Group by Category and summarize
category_summary = (
    df.groupby('Category')
      .agg(Total_Spend=('Spend', 'sum'),
           Transactions=('Spend', 'count'))
      .reset_index()
)

# Calculate percent of total spend
total_spend = category_summary['Total_Spend'].sum()
category_summary['Percent_of_Total'] = 100 * category_summary['Total_Spend'] / total_spend

# Reorder columns and sort
category_summary = category_summary[['Category', 'Total_Spend', 'Percent_of_Total', 'Transactions']]
category_summary = category_summary.sort_values(by='Total_Spend', ascending=False)

# Format amounts and percentages for easier reading (optional)
category_summary['Total_Spend'] = category_summary['Total_Spend'].map('${:,.2f}'.format)
category_summary['Percent_of_Total'] = category_summary['Percent_of_Total'].map('{:.2f}%'.format)

# Display the summary
display(category_summary)


,Category,Total_Spend,Percent_of_Total,Transactions
3,Dining,"$12,931.76",17.53%,916
11,Insurance,"$10,583.75",14.35%,51
6,Electronics,"$10,048.91",13.62%,4
26,Travel/Cruises,"$5,938.28",8.05%,7
28,Travel/Hotel,"$4,748.50",6.44%,5
14,Shopping,"$3,735.06",5.06%,51
9,Groceries,"$3,628.35",4.92%,80
25,Travel/Airlines,"$2,958.84",4.01%,6
31,Utility/Internet,"$1,873.00",2.54%,16
18,Subscription/Cloud Storage,"$1,756.64",2.38%,36


In [30]:
# List all subscription-related categories (as before)
subscription_categories = [
    "Subscription/Cloud Storage",
    "Subscription/AI/Software",
    "Subscription/Patreon",
    "Subscription/Apps",
    "Subscription/Video Streaming",
    "Subscription/Social Media/News"
]

# Filter subscription data
subs_df = df[df['Category'].isin(subscription_categories)]

# Compute total subscription spend
total_subs_spend = subs_df['Spend'].sum()

# Summarize subscription spend by category
subs_summary = (
    subs_df.groupby('Category')
    .agg(Total_Spend=('Spend', 'sum'),
         Transactions=('Spend', 'count'))
    .reset_index()
)

# Compute percentage of subscription total
subs_summary['Percent_of_Subscriptions'] = 100 * subs_summary['Total_Spend'] / total_subs_spend

# Formatting for display
subs_summary['Total_Spend'] = subs_summary['Total_Spend'].map('${:,.2f}'.format)
subs_summary['Percent_of_Subscriptions'] = subs_summary['Percent_of_Subscriptions'].map('{:.2f}%'.format)

# Sort by spend descending
subs_summary = subs_summary.sort_values('Total_Spend', ascending=False)

display(subs_summary)


,Category,Total_Spend,Transactions,Percent_of_Subscriptions
0,Subscription/AI/Software,$678.17,30,21.89%
3,Subscription/Patreon,$673.02,57,21.73%
1,Subscription/Apps,$633.00,6,20.43%
2,Subscription/Cloud Storage,$542.37,41,17.51%
5,Subscription/Video Streaming,$363.77,23,11.74%
4,Subscription/Social Media/News,$207.42,12,6.70%


In [32]:
# Always use the absolute value of Amount for spend analysis
subs_df = df[df['Category'].str.contains('Subscription')].copy()
subs_df['Amount'] = subs_df['Amount'].abs()


In [33]:
vendor_summary = (
    subs_df
    .groupby(['Description', 'Category'], as_index=False)
    .agg(Total_Spend=('Amount', 'sum'), Transactions=('Amount', 'count'))
    .sort_values('Total_Spend', ascending=False)
)


In [34]:
total_subscription_spend = vendor_summary['Total_Spend'].sum()
vendor_summary['Percent_of_Subscriptions'] = (
    vendor_summary['Total_Spend'] / total_subscription_spend * 100
).round(2)
display(vendor_summary)


,Description,Category,Total_Spend,Transactions,Percent_of_Subscriptions
64,GOOGLE *YouTube TV 09/22 PURCHASE g.co/helppay...,Subscription/Cloud Storage,279.00,1,5.08
148,PADDLE.NET* BGDATABALL 05/16 PURCHASE New York NY,Subscription/Apps,180.00,1,3.27
201,SL.NORD* VPNCO 06/30 PURCHASE NEW YORK NY,Subscription/AI/Software,149.88,1,2.73
147,PADDLE.NET* BGDATABALL 04/26 PURCHASE PADDLE.C...,Subscription/Apps,146.00,2,2.66
109,MICROSOFT*MICR 07/22 PURCHASE REDMOND WA,Subscription/AI/Software,129.99,1,2.37
...,...,...,...,...,...
12,APPLE COM BILL 07/07 PURCHASE CUPERTINO CA,Subscription/Cloud Storage,0.99,1,0.02
13,APPLE COM BILL 11/07 PURCHASE CUPERTINO CA,Subscription/Cloud Storage,0.99,1,0.02
19,APPLE.COM/BILL 06/07 PURCHASE 866-712-7753 CA,Subscription/Cloud Storage,0.99,1,0.02
16,APPLE.COM/BILL 02/07 PURCHASE 866-712-7753 CA,Subscription/Cloud Storage,0.99,1,0.02


In [50]:
vendor_map = [
    (r'APPLE COM BILL|APPLE\.COM', 'Apple'),
    (r'SL\.NORD\* VPNCO|NORDVPN', 'NordVPN'),
    (r'GOOGLE\*YOUTUBEPREMIUM|YOUTUBE TV', 'YouTube/Google'),
    (r'PADDLE\.NET\* BGDATABALL', 'Paddle'),
    (r'MICROSOFT', 'Microsoft'),
    (r'HULU', 'Hulu'),
    (r'NETFLIX', 'Netflix'),
    (r'DISNEYPLUS', 'Disney+'),
    (r'AUDIBLE', 'Audible'),
    (r'PATREON', 'Patreon'),
    (r'MEDIUM', 'Medium'),
    (r'LINUX NEW MEDIA', 'Linux New Media'),
    (r'WOLFRAM(ALP|ALPHA)', 'WolframAlpha'),
    (r'MYFITNESSPAL', 'MyFitnessPal'),
    (r'MDN PLUS|MOZILLA', 'Mozilla'),
    (r'GOOGLE.*YOUTUBE', 'YouTube/Google'),
    (r'CRITERIONCHANNEL', 'Criterion'),
    (r'RUMBLE USA', 'Rumble'),
    (r'AUDIBLE', 'Audible'),
    (r'PADDLE\.NET', 'Paddle'),
    (r'MEDIUM', 'Medium'),
    (r'DISNEYPLUS', 'Disney+'),
    (r'HULU', 'Hulu'),
    (r'REMARKABLE', 'Remarkable'),
    (r'HELP\.MAX\.COM|HELP\.HBOMAX\.COM', 'Max/HBOMax'),
    (r'OPENAI.*CHATGPT', 'OpenAI/ChatGPT'),
    (r'ADOBE |ADOBE\.COM|Adobe Inc|Adobe Int', 'Adobe'),
    (r'GOOGLE \*Ground News', 'Ground News'),
    (r'GOOGLE\s*QFIT|GOOGLE ONE|G\.CO/HELPPAY', 'Google'),
    (r'DISNEY PLUS|Disney Plus', 'Disney+'),

]


In [51]:

def extract_vendor(description):
    for pattern, vendor in vendor_map:
        if re.search(pattern, description, re.IGNORECASE):
            return vendor
    return "Other"  # Default if no match


In [52]:
vendor_summary['Vendor'] = vendor_summary['Description'].apply(extract_vendor)


In [53]:
vendor_rollup = (
    vendor_summary
    .groupby('Vendor', as_index=False)
    .agg(Total_Spend=('Total_Spend', 'sum'), Transactions=('Transactions', 'sum'))
)
vendor_rollup['Percent_of_Subscriptions'] = (
    vendor_rollup['Total_Spend'] / vendor_rollup['Total_Spend'].sum() * 100
).round(2)
display(vendor_rollup.sort_values('Total_Spend', ascending=False))


,Vendor,Total_Spend,Transactions,Percent_of_Subscriptions
22,YouTube/Google,1498.70,30,27.27
18,Patreon,673.02,57,12.25
17,Paddle,435.00,4,7.91
14,Netflix,410.83,17,7.47
16,OpenAI/ChatGPT,340.00,17,6.19
0,Adobe,326.83,17,5.95
8,Linux New Media,199.90,2,3.64
21,WolframAlpha,198.00,2,3.60
13,MyFitnessPal,159.98,2,2.91
1,Apple,157.54,22,2.87


In [54]:
# Filter all 'Other' vendors
other_details = vendor_summary[vendor_summary['Vendor'] == 'Other']

# Aggregate by description (to spot patterns)
other_grouped = (
    other_details
    .groupby('Description', as_index=False)
    .agg(Total_Spend=('Total_Spend', 'sum'), Transactions=('Transactions', 'sum'))
    .sort_values('Total_Spend', ascending=False)
)

display(other_grouped)


,Description,Total_Spend,Transactions
